<a href="https://colab.research.google.com/github/3bdillahiomar/3bdillahiomar/blob/main/00_Beledweyne_Flood_Somalia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **FLOOD DETECTION IN BELEDWEYNE, SOMALIA 2023**

Reference: https://github.com/ITC-Water-Resources/itc-hydrafloods-2024

This updated notebook has been optimized for Windows users, addressing compatibility issues present in the older version. Enhancements ensure smoother execution and broader accessibility on Windows platforms.

In [ ]:
## Install libraries

In [42]:
#!pip install ee
#!pip install geemap
#!pip install hydrafloods

In [41]:
import ee
import geemap
import hydrafloods as hf

In [1]:
import ee

In [2]:
ee.Authenticate()

In [3]:
ee.Initialize(project='ee-3bdillahi')

In [4]:
# Beledweyne town somalia
somalia_flood = ee.Geometry.Polygon([
        [44.85, 4],
        [45.85, 4],
        [45.85, 5],
        [44.85, 5]
    ])


In [7]:
print(somalia_flood)
# polygon geometry study area

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              44.85,
              4
            ],
            [
              45.85,
              4
            ],
            [
              45.85,
              5
            ],
            [
              44.85,
              5
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})


In [8]:
#Define time period of interest
start_date = "2023-11-15"
end_date = "2023-12-15"

print(start_date, end_date)

2023-11-15 2023-12-15


## OPTICAL IMAGE ANALYSIS

In [11]:
#Acquire optical data using hf.dataset
optical = hf.Landsat8(somalia_flood, start_date, end_date)
print(optical)

HYDRAFloods Dataset:
{'asset_id': 'LANDSAT/LC08/C02/T1_L2',
 'end_time': '2023-12-15',
 'name': 'Landsat8',
 'region': [[[...], [...], [...], [...], [...]]],
 'start_time': '2023-11-15'}


In [12]:
# function to rename the sun angle properties to what is expected by
# the illumination_correction function
def rename_sunangle_properties(image):
    sz = ee.Number(90).subtract(image.get('SUN_ELEVATION'))
    sa = image.get('SUN_AZIMUTH')
    return image.set({"SOLAR_ZENITH_ANGLE": sz, "SOLAR_AZIMUTH_ANGLE": sa})

# apply the function to rename properties
optical = optical.apply_func(rename_sunangle_properties)

In [14]:
# https://www.youtube.com/watch?v=uCzNxG4Q9aI
elv = ee.Image("JAXA/ALOS/AW3D30/V2_2").select("AVE_DSM").unmask(0) #JAXA ALOS DEM
print(elv)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.unmask",
    "arguments": {
      "input": {
        "functionInvocationValue": {
          "functionName": "Image.select",
          "arguments": {
            "bandSelectors": {
              "constantValue": [
                "AVE_DSM"
              ]
            },
            "input": {
              "functionInvocationValue": {
                "functionName": "Image.load",
                "arguments": {
                  "id": {
                    "constantValue": "JAXA/ALOS/AW3D30/V2_2"
                  }
                }
              }
            }
          }
        }
      },
      "value": {
        "functionInvocationValue": {
          "functionName": "Image.constant",
          "arguments": {
            "value": {
              "constantValue": 0
            }
          }
        }
      }
    }
  }
})


In [16]:
opt_corrected = optical.apply_func(hf.corrections.illumination_correction, elevation = elv, scale=180) #Use JAXA DEM
print(opt_corrected)

HYDRAFloods Dataset:
{'asset_id': 'LANDSAT/LC08/C02/T1_L2',
 'end_time': '2023-12-15',
 'name': 'Landsat8',
 'region': [[[...], [...], [...], [...], [...]]],
 'start_time': '2023-11-15'}


In [19]:
opt_reduced = opt_corrected.collection.median()
# opt_reduced = opt_corrected.collection.mean()
# opt_reduced = opt_corrected.collection.mosaic()
# opt_reduced = opt_corrected.collection.first()
# opt_reduced = opt_corrected.collection.last()

#print(opt_reduced) # median reducer, why?

In [21]:
# Optical thresholding

mndwi = opt_corrected.apply_func(hf.add_indices, indices = ["mndwi"])

mndwi_reduced = mndwi.collection.median()

opt_water = hf.edge_otsu(
    mndwi_reduced,
    "mndwi",
    initial_threshold = 0,
    thresh_no_data = 0.5,
    edge_buffer = 300,
    scale = 1e3,
    invert = True,
    region = somalia_flood
)


In [23]:
import geemap

In [24]:
water_params = {
    "min": 0,
    "max": 1,
    "palette": ['D3D3D3', '000080']
}

Map = geemap.Map(center = (5, 46.5), zoom = 9)

Map.addLayer(somalia_flood, {}, 'Beledweyne, Somalia')
Map.addLayer(opt_water.selfMask(), water_params, 'Maximum Flood Water in Beledweyne, Somalia 2023 (using Optical imagery)')

Map.addLayerControl()
Map

Map(center=[5, 46.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

## SAR IMAGE ANALYSIS

In [27]:
sar = hf.Sentinel1(somalia_flood, start_date, end_date)

In [28]:
from hydrafloods import corrections

In [29]:
sar_speckle_filt = sar.apply_func(hf.gamma_map)
sar_corrected = sar_speckle_filt.apply_func(corrections.slope_correction, elevation = elv, buffer = 30)

In [30]:
sar_reduced = sar_corrected.collection.max()

In [31]:
sar_water = hf.edge_otsu(
    sar_reduced,
    "VV",
    initial_threshold = -11, # Play around with the threshold values a bit?
    thresh_no_data = -20,
    edge_buffer = 300,
    region = somalia_flood,
    scale = 1e3)

In [34]:
water_params = {
    "min": 0,
    "max": 1,
    "palette": ['D3D3D3', '000080']
}

Map = geemap.Map(center = (5, 46.5), zoom = 9)

Map.addLayer(somalia_flood, {}, 'Beledweyne, Somalia')
#Map.addLayer(opt_water, water_params, 'Optical-detected Water')
# Map.addLayer(sar_water.selfMask(), water_params, 'Flood Water 2023')
Map.addLayer(sar_water.selfMask(), water_params, 'Maximum Flood Water in Beledweyne, Somalia 2023 (using SAR imagery)')
Map.addLayerControl()
Map

Map(center=[5, 46.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [37]:
water_params_lightblue = {
    "min": 0,
    "max": 1,
    "palette": ['D3D3D3', '0000FF']
}

water_params_darkblue = {
    "min": 0,
    "max": 1,
    "palette": ['D3D3D3', '000080']
}

#https://geemap.org/notebooks/04_split_panel_map/
left_layer = geemap.ee_tile_layer(opt_water.selfMask(), water_params_lightblue, 'Maximum Flood Water in Beledweyne, Somalia 2023 (using Optical imagery)')
right_layer = geemap.ee_tile_layer(sar_water.selfMask(), water_params_darkblue, 'SAR-detected Water')

Map = geemap.Map(center = (5, 46.5), zoom = 9)
Map.split_map(left_layer, right_layer)
Map.addLayer(somalia_flood, {}, 'Region of Interest')
Map

Map(center=[5, 46.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [39]:
### Abdillahi Osman Omar